<a href="https://colab.research.google.com/github/seojeongbin/web1/blob/master/%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 오버피팅, 언더피팅에 대한 기준은 무엇인지 궁금하다
* 수치적인 기준값이 있는걸까?
* **학습오차와 예측오차를 비교해서 학습오차에 비해 예측오차가 크면 이런게 오버피팅이지않을까?** 


---
* 논의한 랜덤포레스트와, 현부형님께서 소개해주신 모델3가지를 공부 및 나름 적용해보았습니다. 

# 모델링 준비
* 랜덤포레스트
* XG부스트(그레디언트 부스트)
* CV 스태킹
* light GBM 


## 알고리즘 적용을 위한 데이터 준비 

### 데이터 호출 

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
data = pd.read_csv("대회데이터.csv", encoding='cp949')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,5,6,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns = data.iloc[0,:]
data = data.iloc[1:,:]
data = data.reset_index(drop = True)
data = data.reset_index(drop = False)

data

,index,작업번호,마킹길이,절단길이,철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간,절단시간
0,0,1,46,21.9,30,DZB,1,23,PL233,194,827
1,1,2,33.9,111.8,12,A,46,23,PL233,324,2906
2,2,3,49.5,33.5,13,A,2,23,PL234,241,1003
3,3,4,75.7,174.9,13,A,35,23,PL233,483,4412
4,4,5,66.5,66.6,13,A,7,23,PL233,331,1601
...,...,...,...,...,...,...,...,...,...,...,...
162320,162320,162337,139.8,162,19,AH32,31,23,PL231,827,2899
162321,162321,162338,18.5,28.9,16,A,56,25,PL252,165,930
162322,162322,162339,63.7,25.6,15,AH32,1,26,PL261,345,696
162323,162323,162340,94.7,61.4,19,AH32,2,26,PL261,0,2155


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162325 entries, 0 to 162324
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   index   162325 non-null  int64 
 1   작업번호    162325 non-null  object
 2   마킹길이    162325 non-null  object
 3   절단길이    162325 non-null  object
 4   철판두께    162325 non-null  object
 5   철판재질    162325 non-null  object
 6   절단갯수    162325 non-null  object
 7   작업장     162325 non-null  object
 8   절단장비    162325 non-null  object
 9   마킹시간    162325 non-null  object
 10  절단시간    162325 non-null  object
dtypes: int64(1), object(10)
memory usage: 13.6+ MB


In [5]:
del data['index']

In [6]:
data.작업번호 = data.작업번호.astype(float)
data.마킹길이 = data.마킹길이.astype(float)
data.절단길이 = data.절단길이.astype(float)
data.절단갯수 = data.절단갯수.astype(float)
data.마킹시간 = data.마킹시간.astype(float)
data.절단시간 = data.절단시간.astype(float)

### 데이터 전처리 

#### 아웃라이어 제거

In [7]:
from collections import Counter
def detect_outliers(df,features):
  outlier_indices = [] 
  for c in features:
    Q1 = np.percentile(df[c],25)
    Q3 = np.percentile(df[c],75)
    IQR = Q3 - Q1
    outlier_step = IQR * 1.5
    outlier_list_col = df[(df[c] < Q1 - outlier_step) | (df[c] > Q3 + outlier_step)].index #filtre
    outlier_indices.extend(outlier_list_col)
  outlier_indices = Counter(outlier_indices)
  multiple_outliers = list(i for i, v in outlier_indices.items() if v > 2) 
  return multiple_outliers

In [8]:
data.loc[detect_outliers(data,['마킹시간','절단시간'])]

,작업번호,마킹길이,절단길이,철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간,절단시간


* 하나도 탐색되지 않는게 이상합니다(?)
* 아웃라이어 탐색시 칼럼값들을 타겟값만 사용하나요? 계절학기때 배운 강의에 교수님은 다 쓰길래 다 써보겠습니다

In [9]:
data.loc[detect_outliers(data,['마킹시간','절단시간', '마킹길이','절단갯수'])] # 범주형, 작업번호 제외

,작업번호,마킹길이,절단길이,철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간,절단시간
63,64.0,214.4,184.2,14.5,A,64.0,21,PL212,1708.0,5381.0
75,76.0,28.1,194.6,15,A,66.0,21,PL211,1990.0,6729.0
311,312.0,63.2,418.0,19,AH32,114.0,96,PL961,1422.0,23506.0
459,460.0,64.3,289.4,13,A,97.0,26,PL261,1103.0,7231.0
608,609.0,185.3,51.0,20,AH32,2.0,21,PL211,1758.0,5578.0
...,...,...,...,...,...,...,...,...,...,...
161549,161566.0,198.2,47.1,20,AH32,2.0,24,PL242,1047.0,7713.0
161672,161689.0,109.7,211.9,14,AH32,49.0,25,PL254,1062.0,13082.0
161930,161947.0,109.7,186.1,30,AH36,50.0,24,PL241,1276.0,10855.0
5659,5660.0,207.9,371.7,15,A,56.0,96,PL961,24.0,9554.0


* 783개 탐색되었습니다. 

In [10]:
data = data.drop(detect_outliers(data,['마킹시간','절단시간', '마킹길이','절단갯수']))
data

,작업번호,마킹길이,절단길이,철판두께,철판재질,절단갯수,작업장,절단장비,마킹시간,절단시간
0,1.0,46.0,21.9,30,DZB,1.0,23,PL233,194.0,827.0
1,2.0,33.9,111.8,12,A,46.0,23,PL233,324.0,2906.0
2,3.0,49.5,33.5,13,A,2.0,23,PL234,241.0,1003.0
3,4.0,75.7,174.9,13,A,35.0,23,PL233,483.0,4412.0
4,5.0,66.5,66.6,13,A,7.0,23,PL233,331.0,1601.0
...,...,...,...,...,...,...,...,...,...,...
162320,162337.0,139.8,162.0,19,AH32,31.0,23,PL231,827.0,2899.0
162321,162338.0,18.5,28.9,16,A,56.0,25,PL252,165.0,930.0
162322,162339.0,63.7,25.6,15,AH32,1.0,26,PL261,345.0,696.0
162323,162340.0,94.7,61.4,19,AH32,2.0,26,PL261,0.0,2155.0


#### 정규화
* 항목별로 값 스케일 차이가 좀 나는 것 같아서 정규화 해봤습니다

In [11]:
data_a = data[['작업번호', '마킹길이', '절단길이','철판두께','절단갯수','마킹시간','절단시간']]

In [12]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler().fit(data_a)
output = minmax.fit_transform(data_a)
data_scaler = pd.DataFrame(data=output, columns=data_a.columns)
data_scaler.head()

,작업번호,마킹길이,절단길이,철판두께,절단갯수,마킹시간,절단시간
0,0.000000,0.082118,0.033538,0.705882,0.000000,0.000032,0.002375
1,0.000006,0.058903,0.178491,0.176471,0.108434,0.000053,0.008344
2,0.000012,0.088833,0.052241,0.205882,0.002410,0.000039,0.002880
3,0.000018,0.139102,0.280232,0.205882,0.081928,0.000078,0.012669
4,0.000025,0.121450,0.105611,0.205882,0.014458,0.000054,0.004597


In [13]:
data_b = data[['철판재질','절단장비','작업장']]
data = pd.concat([data_b,data_scaler],axis=1)
data.head()

,철판재질,절단장비,작업장,작업번호,마킹길이,절단길이,철판두께,절단갯수,마킹시간,절단시간
0,DZB,PL233,23,0.000000,0.082118,0.033538,0.705882,0.000000,0.000032,0.002375
1,A,PL233,23,0.000006,0.058903,0.178491,0.176471,0.108434,0.000053,0.008344
2,A,PL234,23,0.000012,0.088833,0.052241,0.205882,0.002410,0.000039,0.002880
3,A,PL233,23,0.000018,0.139102,0.280232,0.205882,0.081928,0.000078,0.012669
4,A,PL233,23,0.000025,0.121450,0.105611,0.205882,0.014458,0.000054,0.004597


#### 범주형 데이터 인코딩 

In [14]:
onehot = pd.get_dummies(data, columns = ['철판재질','작업장','절단장비'])
data = pd.concat([data, onehot], axis=1)
data

,철판재질,절단장비,작업장,작업번호,마킹길이,절단길이,철판두께,절단갯수,마킹시간,절단시간,작업번호,마킹길이,절단길이,철판두께,절단갯수,마킹시간,절단시간,철판재질_32M60C,철판재질_460F32MS,철판재질_9NI,철판재질_A,철판재질_AA,철판재질_AAH32,철판재질_AAH32-TM,철판재질_AAH36,철판재질_AE,철판재질_AEH32-TM,철판재질_AH32,철판재질_AH32-TM,철판재질_AH32TL,철판재질_AH36,철판재질_AH36-TM,철판재질_AH36TL,철판재질_AH40-TM,철판재질_B,철판재질_BA,철판재질_BAH32,철판재질_BAH36,철판재질_BZ,철판재질_BZB,...,작업장_21,작업장_22,작업장_23,작업장_24,작업장_25,작업장_26,작업장_27,작업장_91,작업장_92,작업장_96,작업장_97,절단장비_PL141,절단장비_PL142,절단장비_PL211,절단장비_PL212,절단장비_PL221,절단장비_PL223,절단장비_PL224,절단장비_PL231,절단장비_PL233,절단장비_PL234,절단장비_PL241,절단장비_PL242,절단장비_PL252,절단장비_PL253,절단장비_PL254,절단장비_PL261,절단장비_PL262,절단장비_PL271,절단장비_PL272,절단장비_PL273,절단장비_PL911,절단장비_PL921,절단장비_PL922,절단장비_PL961,절단장비_PL973,절단장비_PL974,절단장비_PS213,절단장비_PS243,절단장비_PS963
0,DZB,PL233,23,0.000000,0.082118,0.033538,0.705882,0.000000,0.000032,0.002375,0.000000,0.082118,0.033538,0.705882,0.000000,0.000032,0.002375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A,PL233,23,0.000006,0.058903,0.178491,0.176471,0.108434,0.000053,0.008344,0.000006,0.058903,0.178491,0.176471,0.108434,0.000053,0.008344,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A,PL234,23,0.000012,0.088833,0.052241,0.205882,0.002410,0.000039,0.002880,0.000012,0.088833,0.052241,0.205882,0.002410,0.000039,0.002880,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A,PL233,23,0.000018,0.139102,0.280232,0.205882,0.081928,0.000078,0.012669,0.000018,0.139102,0.280232,0.205882,0.081928,0.000078,0.012669,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A,PL233,23,0.000025,0.121450,0.105611,0.205882,0.014458,0.000054,0.004597,0.000025,0.121450,0.105611,0.205882,0.014458,0.000054,0.004597,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162320,AH32,PL231,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
162321,A,PL252,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
162322,AH32,PL261,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
162323,AH32,PL261,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


### 모델링을 위한 변수할당 

In [15]:
# 마킹시간 예측
y1 = data['마킹시간']
x1 = data.drop(['작업번호', '마킹시간'], axis=1) # 원하는 항목 지워서 x 넣기
# 절단시간 예측
y2 = data['절단시간']
x2 = data.drop(['작업번호', '절단시간'], axis=1) # 원하는 항목 지워서 x 넣기

In [16]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(x1,y1)
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(x2,y2)

In [17]:
print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)

(121741, 145)
(40581, 145)
(121741, 2)


# 모델링 적용
* model = model명()
* model.fit(X_train,y_train)
* 평가방법 rmse 이용 

In [18]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor


## 랜덤포레스트 

In [19]:
model = RandomForestRegressor()
model.fit(X_train1, y_train1)
model.fit(X_train2, y_train2)

ValueError: ignored

* 학습이 안됩니다 ㅜㅜㅜㅜ 

In [ ]:
'''
rf_predict1 = model.predict(X_tes1)
print(f"마킹시간rmse : {np.sqrt(mean_squared_error(y_test1, rf_predict1))}")
model.fit(X2_train, y2_train)
rf_predict2 = model.predict(X_test2)
print(f"절단시간rmse : {np.sqrt(mean_squared_error(y_test2, rf_predict2))}")
'''

## 에이다부스트
* 이전 모델의 과소적합 하였던 훈련 샘플에 가중치 적용

In [20]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import Lasso

model = AdaBoostRegressor(base_estimator=Lasso())

model.fit(X_train1, y_train1)

print(model.score(X_train1, y_train1)) 

print(model.score(X_test1, y_test1))
# max_depth가 1로서 결정노드하나와 리프노드두개로 이루어진트리

ValueError: ignored

* **안됩니다..ㅜ**

* 에이다부스트 앙상블이 과대적합되면 추정기 수를 줄이거나 추정기의 규제를더 강하게 주면된다고 한다
* 좀더 알아보기 
* 근데 부스팅은 알고리즘 여러개 있어야 되는거 아닌가? ? ? 

## 그레디언트 부스트(XG부스트) 

In [21]:
from xgboost import XGBRegressor

model = XGBRegressor()
model.fit(X_train1, y_train1, verbose=False)

print(model.score(X_train1, y_train1)) 

print(model.score(X_test1, y_test1))


ValueError: ignored

* **안됩니다**

## CV스태킹 

* 위의 모델들의 결과물을 입력으로 사용하려 했는데 다 안되니까 슬픕니다.. 

## light GBM 

In [ ]:
from lightgbm.sklearn import LGBMRegressor
import lightgbm as lgb

train_ds = lgb.Dataset(X_train1, label = y_train1) 
test_ds = lgb.Dataset(X_test1, label = y_test1) 


params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)


predict_train = model.predict(X_train1)
predict_test = model.predict(X_test1)

* **안됩니다**